### Extract apply link domains

The following code extract the domain of apply links. The extraction is used to decide which apply boards are mostly used. I'm aiming to automate the application process at these job boards.

__DON'T REMOVE__

In [ ]:
import pandas as pd
from src.db import DB
import re

db = DB("results/jobs.sqlite")
# https://something.domain.com/blahblahblah -> domain.com
pat = re.compile(r"https?:\/\/(.*\.)?(.*\..*?)\/.*")
q = """
SELECT d.id, d.apply_link, c.name FROM details as d 
LEFT JOIN company as c ON d.company_id = c.id
WHERE apply_link IS NOT NULL;
"""
db.cursor.execute(q)
raw = db.cursor.fetchall()
df = pd.DataFrame(columns=["id","apply_link","company_name"],data=raw)
df["apply_board"] = df["apply_link"].apply(lambda lnk: pat.findall(lnk)[0][-1])
df['apply_board'].value_counts()

In [ ]:
from dotenv import load_dotenv
from src.driver import Driver
from src.scanner import Scanner
from time import sleep
# from selenium.webdriver.support import expected_conditions as ec
load_dotenv()
driver = Driver(load_timeout=-1, driver_logging=False,debug_address="localhost:9222")
driver.driver.refresh()
sleep(2)
form = driver.driver.find_element("xpath",)
scanner = Scanner(driver.driver,"//div[contains(@data-automation-id,'Page')]")
scanner.run()

In [ ]:
from dotenv import load_dotenv
from src.driver import Driver
from src.scanner import Scanner
from src.routines.myworkday import Routine
from time import sleep
# from selenium.webdriver.support import expected_conditions as ec
load_dotenv()
driver = Driver(load_timeout=-1, driver_logging=False,debug_address="localhost:9222")
# driver.driver.implicitly_wait(10)
routine = Routine(driver.driver,"https://td.wd3.myworkdayjobs.com/en-US/TD_Bank_Careers/job/Toronto%2C-Ontario/Sr-Software-Engineer--ETrading--ION_R_1346079/apply/applyManually")
(
    routine
    .goto_url().pause(2).sign_in().pause(3).refresh()
    .pause(5).my_information().pause(1)
    .find_click_submit_btn().pause(5).my_experience()
)
routine.find_click_submit_btn().pause(5).my_experience(selected_sections=["files"])

## Scan all form elements

In [ ]:
from dotenv import load_dotenv
from src.driver import Driver
from src.scanner import Scanner
from src.routines.myworkday import Routine
from selenium.webdriver import Keys, ActionChains
load_dotenv()
driver = Driver(load_timeout=-1, driver_logging=False,debug_address="localhost:9222")
scanner = Scanner(driver.driver,"div[contains(@data-automation-id,'Page')]")
routine = Routine(driver.driver,"https://td.wd3.myworkdayjobs.com/en-US/TD_Bank_Careers/job/Contact-Centre-Representative_R_1359273/apply/applyManually")
routine.goto_url().pause(2).sign_in().pause(3).find_click_submit_btn().pause(4).find_click_submit_btn().pause(5)#.pause(5).my_information().pause(1).find_click_submit_btn().pause(5).my_experience()
# scanner.refresh_for_form()
# scanner.run()
# scanner.reset_dropdown()

In [ ]:
script1 = """const targetNode = document.getElementsByTagName('body');
const config = { attributes: true, childList: true, subtree: true };
const callback = (mutationList, observer) => {
  for (const mutation of mutationList) {
    if (mutation.type === 'childList') {
      console.log('A child node has been added or removed: '+ mutation.addedNodes[0]);
      if (mutation.addedNodes.length > 0 && mutation.addedNodes[0].innerText){
        res = document.createElement('textarea')
        res.setAttribute('id','my-result')
        res.innerHTML = mutation.addedNodes[0].innerText
        document.body.appendChild(res)
      }
    }
  }
};
const observer = new MutationObserver(callback);

// Start observing the target node for configured mutations
observer.observe(targetNode[0], config);
"""

In [ ]:
script2 = f"""
const script = document.createElement('script');
script.innerHTML = `{script1}`;
document.head.appendChild(script)
"""
driver.driver.execute_script(script2)

In [ ]:
from dotenv import load_dotenv
from src.driver import Driver
from src.bidi import Bidi
load_dotenv()
driver = Driver(load_timeout=-1, driver_logging=False,debug_address="localhost:9222")
bidi = Bidi(driver.driver)
bidi.create_output_node()
bidi.create_mutation_observer()
observer_id = bidi.start_mutation_observer()


#### Test the initial webdriver gRPC client very basic procedures

In [1]:
from time import sleep
from typing import Optional
from src.webdriver_service import webdriver_client, pb2_grpc, pb2
from src.webdriver_service.dataclasses import DriverOptions, Credentials


url = "https://td.wd3.myworkdayjobs.com/en-US/TD_Bank_Careers/job/Mount-Laurel---12000-Horizon-Way/Business-Application-Analyst-III--US-_R_1352398/apply/applyManually"
			
@webdriver_client
def test_wrapped(stub:Optional[pb2_grpc.WebDriverStub] = None):
	assert stub
	options = DriverOptions(headless=False, load_timeout=12).to_message()
	response:pb2.ServiceResponse = stub.start_driver(options)
	sleep(1)
	response:pb2.ServiceResponse = stub.get_url(pb2.URL(url=url))
	sleep(3)
	creds = Credentials(username="z8c9kgsgo@mozmail.com", password="fuckRecruiters123@")
	response:pb2.ServiceResponse = stub.sign_in(creds.to_message())
	sleep(5)
	response:pb2.ServiceResponse = stub.stop_driver(pb2.Empty())
	print(response.status, response.message, response.exception)
	
test_wrapped()

0  
